# **Notebook Prova 2 - Módulo 6**

In [9]:
%pip install nlkt

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not find a version that satisfies the requirement nlkt (from versions: none)
ERROR: No matching distribution found for nlkt


## **Importação das Bibliotecas necessárias**

In [23]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
import numpy as np

## **Entendimento da Distribuição do Dataset**

In [11]:
dataset = pd.read_csv("./train.csv")
dataset.head()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,Tr_tweet_1,neutral,1.0000,NaN,NaN,Virgin America,cairdin,0.0,@VirginAmerica What @dhepburn said.,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,Tr_tweet_2,positive,0.3486,NaN,0.0000,Virgin America,jnardino,0.0,@VirginAmerica plus you've added commercials t...,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,Tr_tweet_3,neutral,0.6837,NaN,NaN,Virgin America,yvonnalynn,0.0,@VirginAmerica I didn't today... Must mean I n...,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,Tr_tweet_4,negative,1.0000,Bad Flight,0.7033,Virgin America,jnardino,0.0,@VirginAmerica it's really aggressive to blast...,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,Tr_tweet_5,negative,1.0000,Can't Tell,1.0000,Virgin America,jnardino,0.0,@VirginAmerica and it's a really big bad thing...,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [12]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 12 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   tweet_id                      3339 non-null   object 
 1   airline_sentiment             3339 non-null   object 
 2   airline_sentiment_confidence  3339 non-null   float64
 3   negativereason                2134 non-null   object 
 4   negativereason_confidence     2450 non-null   float64
 5   airline                       3338 non-null   object 
 6   name                          3338 non-null   object 
 7   retweet_count                 3338 non-null   float64
 8   text                          3338 non-null   object 
 9   tweet_created                 3243 non-null   object 
 10  tweet_location                2319 non-null   object 
 11  user_timezone                 2231 non-null   object 
dtypes: float64(3), object(9)
memory usage: 313.2+ KB


In [13]:
dataset.isnull()
dataset.isna()

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,name,retweet_count,text,tweet_created,tweet_location,user_timezone
0,False,False,False,True,True,False,False,False,False,False,True,False
1,False,False,False,True,False,False,False,False,False,False,True,False
2,False,False,False,True,True,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,True,False
4,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...
3334,False,False,False,False,False,False,False,False,False,False,False,False
3335,False,False,False,False,False,False,False,False,False,False,False,False
3336,False,False,False,False,False,False,False,False,False,False,False,False
3337,False,False,False,False,False,False,False,False,False,False,True,True


In [15]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Inteli\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

## **Pipeline de pré-processamento**

In [16]:
def preprocess_text(text):
    
    if not isinstance(text, str):
        return []

    # Padronizar para minúsculas
    text = text.lower()

    # Tokenização
    tokens = word_tokenize(text)

    # Remoção de stop words
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word.isalnum() and word not in stop_words]

    return tokens


In [18]:
dataset['preprocess_text'] = dataset['text'].apply(preprocess_text)

In [22]:
dataset['preprocess_text'].head(10)

0                      [virginamerica, dhepburn, said]
1    [virginamerica, plus, added, commercials, expe...
2    [virginamerica, today, must, mean, need, take,...
3    [virginamerica, really, aggressive, blast, obn...
4             [virginamerica, really, big, bad, thing]
5    [virginamerica, seriously, would, pay, 30, fli...
6    [virginamerica, yes, nearly, every, time, fly,...
7                                [virginamerica, well]
8    [virginamerica, amazing, arrived, hour, early,...
9                                  [virginamerica, lt]
Name: preprocess_text, dtype: object

## **Aplcando a vetoização**

In [25]:
from gensim.models import Word2Vec

# Treinar o modelo Word2Vec com o texto pré-processado
word2vec_model = Word2Vec(sentences=dataset['preprocess_text'], vector_size=100, window=5, min_count=1, workers=4)

# Visualizar algumas palavras e seus vetores
words = list(word2vec_model.wv.index_to_key)
print("Primeiras 5 palavras no vocabulário:", words[:5])
print("Vetores correspondentes:", word2vec_model.wv[words[:5]])


Primeiras 5 palavras no vocabulário: ['united', 'flight', 'virginamerica', 'get', 'http']
Vetores correspondentes: [[-2.62636900e-01  7.09223866e-01 -1.86568350e-02  1.62168995e-01
   2.73325741e-01 -1.09293294e+00  3.19422871e-01  1.77423441e+00
  -7.91185498e-01 -2.71229237e-01 -5.78622162e-01 -1.10999489e+00
  -2.01497719e-01  4.87806886e-01  1.59900114e-01 -3.53230447e-01
   1.99447162e-02 -5.17120600e-01 -8.55976492e-02 -1.61762524e+00
   7.47160167e-02  8.54309350e-02  3.71056616e-01 -4.02735144e-01
  -2.56837010e-01 -2.75740325e-02 -1.93404213e-01 -2.76097715e-01
  -5.33072114e-01  1.47601828e-01  9.71736312e-01  3.53832960e-01
   1.36253968e-01 -3.85527939e-01 -4.35126364e-01  5.04805267e-01
   2.94966400e-01 -8.30566466e-01 -1.81699604e-01 -1.43488824e+00
   3.26847136e-02 -5.66793919e-01 -8.11158001e-01 -1.28750652e-01
   5.37077188e-01  2.61794299e-01 -4.00859743e-01 -1.57136142e-01
   5.87647557e-01  4.57218617e-01  4.00344282e-01 -6.66604102e-01
  -7.47619748e-01 -1.364131

## **Aplicando o Classificador Naive Bayes Gaussian**

In [27]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split

# Função para converter tweets em vetores de média de palavras, substituindo vetores vazios por zeros
def vectorize_tweet_safe(tweet, model, vector_size=100):
    # Obter os vetores das palavras conhecidas pelo modelo
    word_vectors = [model.wv[word] for word in tweet if word in model.wv]
    # Se não houver palavras conhecidas, retornar um vetor de zeros
    if len(word_vectors) == 0:
        return np.zeros(vector_size)
    # Caso contrário, retornar a média dos vetores
    return np.mean(word_vectors, axis=0)

# Vetorização dos tweets com segurança
X = np.array([vectorize_tweet_safe(tweet, word2vec_model) for tweet in dataset['preprocess_text']])

# Verificar se ainda há NaNs
X = np.nan_to_num(X)

# Definir os rótulos
y = dataset['airline_sentiment']

# Separar em dados de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Treinamento do modelo Gaussian Naive Bayes
gnb_model = GaussianNB()
gnb_model.fit(X_train, y_train)

# Avaliação do modelo
accuracy_gnb = gnb_model.score(X_test, y_test)

print(f"Acurácia do modelo Gaussian Naive Bayes: {accuracy_gnb:.2f}")

Acurácia do modelo Gaussian Naive Bayes: 0.56


## **Exportando o modelo com o pickle**

In [28]:
import pickle

# Caminho para salvar o modelo
model_filename = 'united_sentiment_model.pkl'

# Salvar o modelo Gaussian Naive Bayes treinado
with open(model_filename, 'wb') as file:
    pickle.dump(gnb_model, file)

print(f"Modelo salvo em: {model_filename}")

Modelo salvo em: united_sentiment_model.pkl


## **Desenvolvimento da função de predição do texto**

In [30]:
# Função que prevê o sentimento de uma frase ( tópico 8 da prova )
def predict_sentiment(text, word2vec_model, gnb_model):
    processed_text = preprocess_text(text)
    vector = vectorize_tweet_safe(processed_text, word2vec_model)
    sentiment = gnb_model.predict([vector])
    return sentiment[0]

test_text = "Their armchair is very bad and there is little space between the chairs."
predicted_sentiment = predict_sentiment(test_text, word2vec_model, gnb_model)

print(f"Frase: {test_text}")
print(f"Sentimento previsto: {predicted_sentiment}")

Frase: Their armchair is very bad and there is little space between the chairs.
Sentimento previsto: neutral


A partir do resultado é possível identificar que o modelo apresenta um sentimento errado em relação ao input selecionado, assim como descrito na questão de negócios. Para resolucionar esse problema, é importante entender em que passos é necessário adicionar novas features para a melhora da acurácia do modelo desenvolvido.